# Subclassing in pySIMTRA

This guide describes how to use subclassing of the pySIMTRA classes to extend the functionality of the package in order to adjust the deposition model to further resemble your sputter system. The same multi-cathode sputter system as described in the guide `Define a Multi-cathode system` will be used here. We will start by defining a custom magnetron, which will later be used in a custom sputter system class. 

As with the other guides, we will start with importing the required packages and to make sure the SIMTRA executable was assigned to the python package:

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from pymatgen.core.periodic_table import Element  # needed to get the density and atomic mass for an element
import pysimtra as ps

# Import the classes which are later subclassed for type hinting
from pysimtra import Magnetron, SputterSystem

In [ ]:
# Define the path to the SIMTRA application folder
sim_path = 'C:/Users/Felix/Desktop/simtra_v2.2'
# Import the application into the package
ps.import_exe(sim_path)

## Define a custom magnetron

First, we define a custom magnetron. This can be handy in a sputter system with multiple identical cathodes. This way, all constant parameters can be conveniently hidden in the class, and only the frequently changed parameters can be exposed via the custom class. In this example, we try to infer the racetrack file from the defined sputtered element. Make sure to replace the path with your local one and make sure that the file exists when replicating this example.

In [4]:
# Define the custom magnetron, subclass the original pySIMTRA class
class CustomMagnetron(Magnetron):

    def __init__(self, name: str, pos: float, tilt: float, elem: str = None, n_particles: int = 10 ** 8):

        """
        Class for a custom magnetron. The class defines a magnetron object based on a reduced number of parameters, e.g. the 
        racetrack path is generated from the element.
        
        :param name: name of the magnetron (ideally in camelCase)
        :param pos: cathode position (in °) on the "anker circle"
        :param tilt: cathode tilt (in °)
        :param elem: element to be sputtered
        :param n_particles: number of particles to simulation, defaults to 10**8
        """

        # Construct the magnetron from circles, cones and cylinders
        target = ps.Circle(name='target', radius=0.0191, position=(0, 0, 0.055))
        shield = ps.Cone(name='shield', small_rho=0.0191, big_rho=0.023, height=0.005, position=(0, 0, 0.055))
        cap = ps.Circle(name='cap', radius=0.03, position=(0, 0, 0.06))
        cap.perforate(by='circle', radius=0.023)
        body = ps.Cylinder(name='body', radius=0.03, height=0.06)
        # Define the anker radius (identical to the multi-cathode guide)
        r_anker = 0.06
        # Calculate the position (in m) and orientation (in °)
        pos = np.cos(np.radians(pos)) * r_anker, np.sin(np.radians(pos)) * r_anker, 0
        orien = pos, -tilt, 0
        # Create the dummy object from the surfaces
        m_object = ps.DummyObject(name=name, surfaces=[target, shield, cap, body], position=pos, orientation=orien)
        # Define a path to the racetrack file
        r_path = 'racetracks/%s_racetrack_1.5_inch.txt' % element
        # Construct the class by calling the superclass
        super().__init__(transported_element=elem, m_object=m_object, n_particles=n_particles, sputter_surface_index=1, 
                         racetrack_file_path=r_path)

## Define a custom sputter system

The same multi-cathode sputter system as described in the `Define a Multi-cathode system` guide. The class will also use the custom magnetron class defined above. Since the sputter system works with mTorr instead of Pa, we will expose the pressure in mTorr to the class and internally convert to SI units to improve useability. Often, sputter systems allow for the adjustment of the cathode tilt via an adjustment screw. Therefore, as an example, we will expose the scale reading to the user and convert to a tilt in ° internally. To keep the tutorial simple, we won't cover complete error handling of inputs here. 

In [ ]:
class CustomSputterSystem(SputterSystem):

    def __init__(self, elements: dict[int, str], output_path: str, tilt: float = 20, pressure: float = 3.7, n_particles: int = 10 ** 8):

        """
        Creates a custom four-cathode sputter system.

        :param elements: materials in the sputtering chamber given by the cathode number (beginning at 1) and the element symbol name
        :param output_path: path at which the simulation results will be stored
        :param tilt: cathode tilt in mm as visible on the screws, defaults to 20 mm which equals 10°
        :param pressure: pressure in mTorr used for sputtering, defaults to 3.7 mTorr
        :param n_particles: number of particles to simulate, defaults to 10**8
        """

        # Create a cylindrical sputter chamber, convert from Pa to mTorr
        chamber = Chamber.cylindrical(radius=0.12, length=0.18, temperature=293.15, pressure=0.13332 * pressure)
        # The chamber will have four cathodes positioned on an "anker circle" and with an identical tilt
        cat_pos = {1: 45, 2: 135, 3: -135, 4: -45}  # °
        # Convert the tilt on the adjustment screw to actual cathode tilt
        t_deg = tilt / 2  # °
        # Create a magnetron for each element
        mags: list[ps.Magnetron] = []
        for i, pos in cat_pos.items():
            # Create the custom magnetron
            mag = CustomMagnetron(name'mag_%d' % i, pos, t_deg, elements[i], n_particles)
            mags.append(mag)
        # Define the substrate which also will be the same for all depositions (identical to the previous guide)
        # Convert the table height parameter to the actual position in the chamber
        s_surf = ps.Rectangle(name='subSurface', dx=0.05, dy=0.05, save_avg_data=True, avg_grid=(21, 21))
        substrate = ps.DummyObject(name='substrate', surfaces=[s_surf], position=(0, 0, 0.2 - table_height))
        # Initialize the superclass
        super().__init__(chamber, mags, substrate, output_path)

    # Custom method for starting a deposition
    def sim_deposition(self, voltages: dict[int, float], dep_rates: dict[int, float]) -> pd.DataFrame:

        """
        Initiates the simulation of the deposition. 

        :param voltages: voltages of the power supplies
        :param dep_rates: measured deposition rates in the center of the substrate
        :return: composition in atomic percent
        """

        # Set the voltages on the power supplies as maximum ion energies
        elements: list[str] = []
        for i, v for voltages.items():
            # Find the magnetron with the respective name and set the maximum ion energy
            mag = [m for m in self.magnetrons if m.name == 'mag%d' % i][0]
            mag.max_ion_energy = v
            # Store the deposited element again for later generating the composition
            elements.append(mag.transported_element)
        # Call the simulate function in the superclass
        sim_res = self.simulate(['mag%d' % i for i in voltages.keys()])
        # Get the number of particles for every magnetron and flatten the result
        n_particles = {i: res.n_particles['substrate'].flatten() for i, res in zip(voltages.keys(), sim_res)}
        # Wrap the results in a pandas DataFrame, change the column names to the elements
        n_particles = pd.DataFrame.from_dict(n_particles, index=range(1, 442), columns=elements)
        # Convert to a deposition profile by normalizing to the center area and multiplying with the deposition rates
        dep_profile = n_particles / n_particles.loc[220] * pd.Series(dep_rates)
        # Calculate the ratio to arrive at the composition in volume percent
        vol = dep_profile.div(dep_profile.sum(axis=1), axis=0) * 100
        # Get the density and the atomic of each element 
        density = pd.Series({e: Element(e).density for e in elements})
        at_mass = pd.Series({e: Element(e).atomic_mass for e in elements})
        # Return the composition
        return (density * vol / at_mass).div(density * volume / at_mass).sum(axis=1), axis=0)

## Run the simulation

In [ ]:
# Define the elements
elements = {1: 'Ni', 2: 'Pd', 3: 'Pt', 4: 'Ru'}
# Define the voltages on the power supplies for every magnetron
voltages = {1: 200, 2: 250, 3: 190, 4: 180}  # V
# Define the deposition rates, for this example we assume that the deposition rates are independent on the angle
dep_rates = {1: 0.04, 2: 0.15, 3: 0.1, 4: 0.05}  # nm/s
# Define an output path for the simulation
out_path = 'C:/Users/Felix/Desktop/sim_result'

# Define the sputter system
system = CustomSputterSystem(elements, output_path, tilt=20, n_particles=10**6)
# Run the deposition simulation and retrieve the composition
composition = system.sim_deposition(voltages, dep_rates)

## Plot the results

In [ ]:
# Create a plot with as many subplots as there are elements
fig, ax = plt.subplots(1, len(elements), figsize=(len(elements) * 5, 4))

# Iterate over the elements and load plot the composition
for elem in elements.values():
    # Get the composition and reshape it to the original grid (the values are originating from the definition of the substrate in the
    # custom sputter system class
    xy = np.linspace(-0.05, 0.05, 21)
    X, Y = np.meshgrid(xy, sy)
    comp = composition[elem].values.reshape(X.shape)
    # Plot the composition
    pc = ax[i].pcolormesh(X, Y, comp)
    # Add axes labels
    ax[i].set_xlabel('x [m]')
    ax[i].set_ylabel('y [m]')
    # Add a title showing the sputtered element
    ax[i].set_title(elem)
    # Add a colorbar
    fig.colorbar(pc, ax=ax[i], label='composition [at. %]')

# Adjust layout and show the plot
fig.tight_layout()
plt.show()